In [2]:
import pickle
import sys
from collections import Counter
import numpy as np
from scipy import spatial
from pandas import DataFrame

sys.path.append("/home/jichao/Desktop/reddit/language_model")
from language_model import *

tokens_men2fem_pre_cp = pickle.load(open("tokens_men2fem_pre_cp.pickle"))
tokens_men2fem_post_cp_men = pickle.load(open("tokens_men2fem_post_cp_men.pickle"))
tokens_men2fem_post_cp_fem = pickle.load(open("tokens_men2fem_post_cp_fem.pickle"))
tokens_men_only = pickle.load(open("tokens_men_only.pickle"))

d = [tokens_men2fem_pre_cp,
    tokens_men2fem_post_cp_men,
    tokens_men2fem_post_cp_fem,
    tokens_men_only]

logprob = np.empty((4, 4))

for i in range(len(d)):
    train = d[i]
    train_tokens = set(list(itertools.chain(*train)))

    for j in range(len(d)):
        if j != i:
            test = d[j]
            test_tokens = list(itertools.chain(*test))

            bigram = Bigram(train, special_token=True)
            oov = filter(lambda token: token not in train_tokens, test_tokens)
            oov = len(set(oov))

            bigram.fit(oov)
            data = map(lambda tokens: bigram.predict(tokens), test)
            logprob[i][j] = np.mean(data)
        else:
            logprob[i][j] = np.nan

logprob = DataFrame(logprob,
                    index=["M2F_PreCP_M", "M2F_PostCP_M", "M2F_PostCP_F", "M_Only"],
                    columns=["M2F_PreCP_M", "M2F_PostCP_M", "M2F_PostCP_F", "M_Only"])
logprob.index.name = "Train"
logprob.columns.name = "Test"
logprob

Test,M2F_PreCP_M,M2F_PostCP_M,M2F_PostCP_F,M_Only
Train,,,,
M2F_PreCP_M,NaN,-6.967476,-6.933751,-6.962005
M2F_PostCP_M,-6.924987,NaN,-6.916690,-6.945968
M2F_PostCP_F,-7.041595,-7.069118,NaN,-7.092645
M_Only,-6.938615,-6.983586,-6.956517,NaN
